In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import nltk
from nltk.util import ngrams
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, Embedding, Dense, Bidirectional

In [ ]:
myfile = '/content/drive/MyDrive/Colab Notebooks/Sentence_Auto_Completion_Using_NLP/holmes.txt'

with open(myfile, 'r', encoding='utf-8') as file:
  text = file.read()

In [ ]:
len(text)

236110709

In [ ]:
data = text[:1000000]

In [ ]:
def clean_text(text):
    # Remove non-alphanumeric characters except spaces
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    return text.strip()  # Strip leading and trailing whitespace

def preprocessed_text(text):
    sentences = text.split('\n')
    cleaned_sentences = [clean_text(sentence) for sentence in sentences if len(sentence.strip()) > 0]
    tokenize = []
    for sentences in cleaned_sentences:
      sentences = sentences.lower()
      tokenize.append(sentences)
    return tokenize

In [ ]:
tokenize_data = preprocessed_text(data)

*OOV Token* => Is used for unknown words in trainning data.

In [ ]:
tokenizer = Tokenizer(oov_token = '<OOV>')
tokenizer.fit_on_texts(tokenize_data)

In [ ]:
tokenizer_len = len(tokenizer.word_counts)

In [ ]:
n_gram_sequence = []
for line in tokenize_data:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram = token_list[: i+1]
    n_gram_sequence.append(n_gram)

In [ ]:
# n_gram_sequence

Padding and List Comprehension

In [ ]:
max_len = max([len(x) for x in n_gram_sequence])
pad_sequences_data = np.array(pad_sequences(n_gram_sequence, maxlen = max_len, padding= 'pre'))

In [ ]:
pad_sequences_data

array([[   0,    0,    0, ...,    0,  209, 1814],
       [   0,    0,    0, ...,  209, 1814,  197],
       [   0,    0,    0, ..., 1814,  197,    5],
       ...,
       [   0,    0,    0, ...,    6,  730,   69],
       [   0,    0,    0, ...,  730,   69,   49],
       [   0,    0,    0, ...,   69,   49,   21]], dtype=int32)

In [ ]:
pad_sequences_data.shape

(156818, 16)

In [ ]:
x_train = pad_sequences_data[:,:-1]
y_train = pad_sequences_data[:,-1]

In [ ]:
x_train

array([[   0,    0,    0, ...,    0,    0,  209],
       [   0,    0,    0, ...,    0,  209, 1814],
       [   0,    0,    0, ...,  209, 1814,  197],
       ...,
       [   0,    0,    0, ..., 1207,    6,  730],
       [   0,    0,    0, ...,    6,  730,   69],
       [   0,    0,    0, ...,  730,   69,   49]], dtype=int32)

In [ ]:
y_train

array([1814,  197,    5, ...,   69,   49,   21], dtype=int32)

In [ ]:
X, labels = x_train, y_train

In [ ]:
def to_categorical_safe(y, num_classes):
    y = np.array(y, dtype='int')

    if np.any(y < 0):
        raise ValueError("Negative values found in `y` after subtracting 1")

    if np.any(y >= num_classes):
        raise ValueError(f"Found an index in `y` that is out of bounds for the array size of {num_classes}")

    return tf.keras.utils.to_categorical(y, num_classes=num_classes)

try:
    categorical_labels = to_categorical_safe(pad_sequences_data, tokenizer_len)
    print(categorical_labels)
except ValueError as e:
    print(e)

Found an index in `y` that is out of bounds for the array size of 9461


In [ ]:
tf.keras.utils.to_categorical(labels-1, num_classes= tokenizer_len)